<center><h1>Cultural Analytics: Homework #2</h1></center>
<center><h2>ENGL 64.05, 21S</h2></center>
<br>
<center><b>Due</b> 11:59PM 04/16/2021</center>

---

In [ ]:
# make sure we're in the home directory
%cd ~

In [ ]:
import os
import csv
import numpy as np
import sklearn
import pandas as pd
from operator import itemgetter

import sys
sys.path.append('shared/engl64.05/lib')
from tsvdro import tsvdro

<h2><font color="Green"> HW 2.1</font>: Vectorization and Comparison by Genre</h2>

<p>You'll need to take a sample of entries from Ted Underwood's metadata and then run through the remainder of the cells in this group and then respond to the prompt.</p>


In [ ]:
# This time we're going to load the CSV using Pandas into a DataFrame.
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html
# 
# Using a DataFrame will have some obvious benefits, including the automatic reading
# of the header row and guessing datatypes for our numerical values (i.e., year of publication)
metadata = pd.read_csv("shared/engl64.05/data/Underwood_ch1/allgenremeta.csv")

<h2><font color="Red">Sample Data</font></h2>

In [ ]:
# Create a sample from the metadata using Pandas and assign it to 
# a variable named "sample." How much data should you sample? 
# What do you think makes sense?
#
# See our readings from Monday and https://pandas.pydata.org for help with this

In [ ]:
# For this example and because the documents provided have already been rendered data
# as word frequencies, we'll use the DictVectorizer. This will simply combine the existing 
# vocabulary.

from sklearn.feature_extraction import DictVectorizer
vectorizer = DictVectorizer()

In [ ]:
doc_data = list()
for filename in [f + '.dro' for f in sample['docid'].tolist()]:
    tmp_data = tsvdro.load("shared/engl64.05/data/Underwood_ch1/" + filename)
    doc_data.append(tmp_data['data'])

In [ ]:
# vectorize from DRO documents
dtm = vectorizer.fit_transform(doc_data)

In [ ]:
# produce a distance matrix using Cosine similarity
# https://en.wikipedia.org/wiki/Cosine_similarity
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

from sklearn.metrics.pairwise import cosine_similarity
cosine_dist_matrix = 1 - cosine_similarity(dtm)

In [ ]:
# The Cosine similarity matrix is the size of sample x sample 
# We need to reduce this to two dimensions in order to plot the 
# data. We want to see if we can separate texts by their genre.

from sklearn.manifold import MDS
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)
pos = mds.fit_transform(cosine_dist_matrix)

In [ ]:
# Visualize
# 
# NOTE: you might need to click on the image to expand it enough to read it. 
# 
# Too many objects to read? Modify your sample. 
# Not enough points? Modify your sample.
# 

import matplotlib.pyplot as plt
xs, ys = pos[:, 0], pos[:, 1]
fig = plt.figure(figsize=(35, 20), dpi=75)
genres = list(sample['genretags'])
authors = list(sample['author'])

# this will loop through four different lists simultaneously
for x, y, genre, author in zip(xs, ys, genres,authors):

    # plot the point itsself
    plt.scatter(x, y, c='lightgrey',s=6)

    # These two lines will annotate the point by displaying 
    # the genre (in red) and author (in blue). 
    plt.text(x, y, genre, c="red", ha='center', va='bottom')
    plt.text(x, y, author, c="blue", ha='center', va='top')

    plt.axis('off')
plt.show()

## Written Response: 
In no more than two hundred words respond to the following prompt: What do you observe in the above figure? What happens when you rerun these cells? How robust are these results? What questions do you have about the dataset and methods?

---
<h2><font color="Green"> HW 2.2</font>: Vectorization of ASCII Text</h2>

Insert a new cell and:
1. Vectorize all of the 150 English-language novels in the Novel450 dataset (review Lab 2 notebook)
2. No stopwords
3. Only include vocabulary that appears in 50% of the texts.
4. Your document-term matrix should be called counts

In [ ]:
# This will give us a set of shorter labels
labels = [os.path.basename(f).replace('.txt','') for f in input_files]

In [ ]:
# calculate cosine similarity
cosine_dist_matrix = 1 - cosine_similarity(counts)

# Display distance from Pride and Prejudice
for x,y in sorted(enumerate(cosine_dist_matrix[5]), key=itemgetter(1)):
    print('{0:.4f} {1}'.format(y,labels[x]))

In [ ]:
# calculate Euclidean distance
from sklearn.metrics import euclidean_distances
euclidean_dist_matrix = euclidean_distances(counts)

# Display distance from Pride and Prejudice
for x,y in sorted(enumerate(euclidean_dist_matrix[5]), key=itemgetter(1)):
    print('{0:.2f} {1}'.format(y,labels[x]))

<h2><font color="Red">Create a Plot</font></h2>
Add a new cell that produces a scatter plot of the cosine similarity data (cosine_dist_matrix). Instead of adding author and genre labels (as above), just annotate with the filename lables in the labels dictionary. You can modify the above code to make this work.

## Written Response: 
In no more than two hundred words respond to the following prompt: What do you observe in the above cells? What differences do you see between the two distance metrics (Cosine similarity and Eucidean distance)? What do you think these methods tell us about modeling semantic space in this novel collection?